In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

teams_dict, teams_list = get_teams(league="NBA")
box_DIR = "../data/box/"
img_DIR_T = "../data/images/teams/"
fig_DIR = "../figs/teams/"

In [ ]:
df0 = get_box("T","Adv",cum=True)
df0 = df0.sort_values("net_rating",ascending=False).reset_index(drop=True)
df0["net_rating_R"] = df0["net_rating"].rank(ascending=False,method="first")

In [ ]:
# df0 = get_box("T","Adv",cum=True)
# df0 = df0.sort_values("net_rating",ascending=False).reset_index(drop=True)
df0["tcat"] = 5
df0.loc[df0.query("net_rating_R<=5").index,"tcat"] = 0
df0.loc[df0.query("net_rating_R<=10 & net_rating_R>5").index,"tcat"] = 1
df0.loc[df0.query("net_rating_R<=15 & net_rating_R>10").index,"tcat"] = 2
df0.loc[df0.query("net_rating_R<=20 & net_rating_R>15").index,"tcat"] = 3
df0.loc[df0.query("net_rating_R<=25 & net_rating_R>20").index,"tcat"] = 4
df0["tcat"] = df0["tcat"].astype("category")

In [ ]:
dfs = pd.read_parquet(box_DIR+"NBA_Box_T_Adv_2024.parquet")
dfs = dfs.drop(dfs.query("netRating == 0").index,axis=0).reset_index(drop=True)
dfs.to_parquet(box_DIR+"NBA_Box_T_Adv_2024.parquet")

In [ ]:
df1 = get_box("T","Adv",cum=False)
df1 = df1.sort_values("netrating",ascending=False).reset_index(drop=True)
df1["ncat"] = 6
df1.loc[df1.query("netrating>=15").index,"ncat"] = 0
df1.loc[df1.query("netrating<15 & netrating>=7").index,"ncat"] = 1
df1.loc[df1.query("netrating<7 & netrating>=0").index,"ncat"] = 2
df1.loc[df1.query("netrating<0 & netrating>-7").index,"ncat"] = 3
df1.loc[df1.query("netrating<=-7 & netrating>-15").index,"ncat"] = 4
df1.loc[df1.query("netrating<=-15").index,"ncat"] = 5
df1["ncat"] = df1["ncat"].astype("category")

In [ ]:
df2 = df1[["gameid","teamid","ncat","netrating"]]
df2 = df2.groupby("gameid")
df31 = pd.merge(df2.nth(0),df2.nth(1),on="gameid",suffixes=["","2"])
df32 = pd.merge(df2.nth(0),df2.nth(1),on="gameid",suffixes=["","2"])


In [ ]:
df4 = pd.merge(df3,df0[["team_id","tcat"]],left_on="teamid2",right_on="team_id")
# df4["team"] = df4["teamid"].map(teams_dict)

In [ ]:
df5 = df4.groupby("teamid")[["ncat","tcat"]].value_counts().reset_index()
df5["team"] = df5["teamid"].map(teams_dict)

In [ ]:
team = "LA Clippers"
dfc = df5.query(f"team == '{team}'")

In [ ]:
dfc

In [ ]:
p = (
            ggplot(dfc,aes(x="ncat", y="tcat", fill="count"))
            + geom_tile(aes(width=.88, height=.88))
            + geom_text(aes(label='count'), size=14, show_legend=False, fontweight = "bold") 
            + scale_fill_gradient(low = "#f2f2f2", high = "red")
            + labs(
                title = team + ": Rating Scorigami",
                # subtitle = f"# NBA Games with different combinations of\n  {var1} & {var2}",
                caption = "shiny.sradjoker.cc/NBA-Scorigami",
                x = "Game Net Rating",
                y = "Opp Net Rating",
            )
            + theme_xkcd(base_size=14, stroke_color="none")
            + theme(
                figure_size= (12,12),
                plot_background = element_rect(fill = 'white', color = "white"),
                legend_position="none",
                plot_title=element_text(face="bold", size=18),
                plot_subtitle=element_text(size=13,weight="normal",linespacing=1.5),
                plot_caption=element_text(vjust=-0.04,hjust=0,size=10),
                axis_text_y = element_text(size = 12, vjust=1),
                text=element_text(family=["Comic Neue"],weight="bold"),
                plot_margin=0.02,
            )
            + theme(
                axis_ticks_major_y=element_blank(),
                axis_ticks_major_x=element_blank(),
                axis_ticks_minor_y=element_blank(),
                axis_ticks_minor_x=element_blank(),
                panel_grid_major_y=element_blank(),
                panel_grid_major_x=element_blank(),
                panel_border=element_blank(),
            )
        )
p